In [1]:
from ophelia.start import Ophelia

In [4]:
ophelia = Ophelia("Ophelia-Test")

13:41:19.970 Ophelia [TAPE] +-----------------------------------------------------------------+
13:41:19.970 Ophelia [INFO] | Hello! This engine is for data mining & ml pipelines in PySpark |
13:41:19.970 Ophelia [INFO] | Welcome to Ophelia Spark miner engine                           |
13:41:19.970 Ophelia [INFO] | Package Version Ophelia.0.0.1                                   |
13:41:19.970 Ophelia [WARN] | V for Vendata...                                                |
13:41:19.970 Ophelia [TAPE] +-----------------------------------------------------------------+
13:41:19.970 Ophelia [WARN]                   - Ophelia a Spark miner -                    
13:41:19.970 Ophelia [MASK]   █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █
13:41:19.970 Ophelia [MASK]   █ █ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ █ █
13:41:19.970 Ophelia [MASK]   █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █
13:41:19.970 Ophelia [MASK]   █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ 

In [3]:
sc = ophelia.Spark.build_spark_context()
spark = ophelia.SparkSession

13:35:50.301 Ophelia [INFO] Spark Context Initialized Success


In [4]:
from ophelia.read.spark_read import Read

In [5]:
sample_bank_df = spark.readFile("data/csv/100sample_bank", "csv")
sample_bank_df.show()

13:56:02.164 Ophelia [INFO] Read CSV File From Path: data/csv/100sample_bank
+---+-------------+-------+---------+-------+-------+--------+--------+
|age|          job|marital|education|housing|balance|duration|poutcome|
+---+-------------+-------+---------+-------+-------+--------+--------+
| 36| entrepreneur|married|secondary|     no|   2971|     247| failure|
| 30|     services| single|  primary|    yes|   5746|     125| unknown|
| 29|  blue-collar| single|secondary|     no|   2806|     294|   other|
| 35|   technician|married|secondary|     no|      0|     125| unknown|
| 30|   management| single| tertiary|     no|    697|     179| success|
| 33|       admin.|married|secondary|     no|      0|     214| unknown|
| 31|   management| single| tertiary|     no|   1308|     323| unknown|
| 55|    housemaid|married|  primary|     no|      4|      85| unknown|
| 59|      retired|married|secondary|     no|    136|     301| unknown|
| 34|self-employed| single|secondary|    yes|      3|     3

In [6]:
from ophelia.functions import DynamicSampling

In [7]:
sample_df = sample_bank_df.simple_sample(10)
sample_df.show()

13:56:10.936 Ophelia [INFO] Ophelia Active Session
+---+-------------+--------+---------+-------+-------+--------+--------+
|age|          job| marital|education|housing|balance|duration|poutcome|
+---+-------------+--------+---------+-------+-------+--------+--------+
| 55|   management|divorced| tertiary|     no|   2037|     305| success|
| 33|   technician| married|secondary|    yes|   2362|     271| unknown|
| 39|   management|  single| tertiary|     no|    633|     105| failure|
| 29|     services| married|secondary|    yes|    363|     164| unknown|
| 54|self-employed| married| tertiary|     no|    368|       7| unknown|
| 52|       admin.| married|secondary|     no|   1113|      83| unknown|
| 32|   management|  single| tertiary|    yes|    974|     228| unknown|
| 27|      student|  single| tertiary|     no|    311|     462| success|
| 33|  blue-collar|  single|secondary|     no|     45|      13| unknown|
| 37|  blue-collar|divorced|secondary|    yes|    719|     118| failure|


In [8]:
from schemas.fake_schemas import bank_schema

In [ ]:
sample_df.coalesce(1).write.mode("overwrite").csv(
    "data/csv/100sample_bank", header=True, sep=","
)

In [10]:
spark.read.csv("data/csv/100sample_bank", schema=bank_schema, header=True).show()

+---+-------------+-------+---------+-------+-------+--------+--------+
|age|          job|marital|education|housing|balance|duration|poutcome|
+---+-------------+-------+---------+-------+-------+--------+--------+
| 36| entrepreneur|married|secondary|     no|   2971|     247| failure|
| 30|     services| single|  primary|    yes|   5746|     125| unknown|
| 29|  blue-collar| single|secondary|     no|   2806|     294|   other|
| 35|   technician|married|secondary|     no|      0|     125| unknown|
| 30|   management| single| tertiary|     no|    697|     179| success|
| 33|       admin.|married|secondary|     no|      0|     214| unknown|
| 31|   management| single| tertiary|     no|   1308|     323| unknown|
| 55|    housemaid|married|  primary|     no|      4|      85| unknown|
| 59|      retired|married|secondary|     no|    136|     301| unknown|
| 34|self-employed| single|secondary|    yes|      3|     301| unknown|
| 37|  blue-collar|married|secondary|    yes|    374|     944| u

In [ ]:
sample_schema = sample_df.schema
spark.read.csv("data/csv/100sample_bank", sample_schema).show()

In [11]:
import pandas as pd
from data.fake_datasets import sales_dataset

In [12]:
df = spark.createDataFrame(pd.DataFrame(data=sales_dataset))
df.show(5)

+-------+----+-------+
|Product|Year|Revenue|
+-------+----+-------+
|      A|2010|    100|
|      B|2010|    200|
|      C|2010|    300|
|      A|2011|    110|
|      B|2011|    190|
+-------+----+-------+
only showing top 5 rows



In [13]:
from ophelia.functions import Shape

df.Shape

(9, 3)

In [14]:
from ophelia.functions import PctChange

df.pctChange().show(5, False)

+-------------------+
|Revenue            |
+-------------------+
|null               |
|1.0                |
|0.5                |
|-0.6333333333333333|
|0.7272727272727273 |
+-------------------+
only showing top 5 rows



In [15]:
df.pctChange(periods=2).na.fill(0).show(5, False)

+--------------------+
|Revenue             |
+--------------------+
|0.0                 |
|0.0                 |
|2.0                 |
|-0.44999999999999996|
|-0.3666666666666667 |
+--------------------+
only showing top 5 rows



In [16]:
df.pctChange(partition_by="Product", order_by="Year", pct_cols="Revenue").na.fill(
    0
).show(5, False)

+---------------------+
|Revenue              |
+---------------------+
|0.0                  |
|-0.050000000000000044|
|0.1578947368421053   |
|0.0                  |
|0.06666666666666665  |
+---------------------+
only showing top 5 rows



In [17]:
df.pctChange(
    partition_by="Product", order_by="Year", pct_cols=["Year", "Revenue"]
).na.fill(0).show(5, False)

+--------------------+---------------------+
|Year                |Revenue              |
+--------------------+---------------------+
|0.0                 |0.0                  |
|4.975124378110429E-4|-0.050000000000000044|
|4.972650422674363E-4|0.1578947368421053   |
|0.0                 |0.0                  |
|4.975124378110429E-4|0.06666666666666665  |
+--------------------+---------------------+
only showing top 5 rows



In [19]:
from ophelia.functions import Reshape

# We can choose either one column to aggregate with one single operation
agg_dict_test = {"Revenue": "max"}

# Or moreTthan one column and more than one operation
_agg_dict_test = {
    "Revenue": "mean, count, sum, stddev, var, min",
    "Year": "count, sum",
    "Product": "count",
}

df.toWide(group_by="Product", pivot_col="Year", agg_dict=agg_dict_test).orderBy(
    "Product"
).show(5, False)

+-------+----+----+----+
|Product|2010|2011|2012|
+-------+----+----+----+
|A      |100 |110 |120 |
|B      |200 |190 |220 |
|C      |300 |320 |350 |
+-------+----+----+----+



In [20]:
from data.fake_datasets import bank_dataset

In [21]:
bank_df = spark.createDataFrame(pd.DataFrame(bank_dataset))
bank_df.show()

+---+----------+--------+---------+-------+-------+-------+----+---+-----+--------+--------+
|age|       job| marital|education|default|balance|housing|loan|day|month|duration|campaign|
+---+----------+--------+---------+-------+-------+-------+----+---+-----+--------+--------+
| 59|    admin.| married|secondary|     no|   2343|    yes| yes|  5|  may|    1042|       1|
| 56|    admin.| married|secondary|    yes|     45|     no|  no|  5|  may|    1467|       1|
| 41|technician| married|secondary|     no|   1270|    yes|  no|  5|  jun|    1389|       1|
| 55|  services| married|secondary|     no|   2476|    yes|  no|  5|  jul|     579|       1|
| 54|    admin.| married| tertiary|    yes|    184|     no|  no|  5|  jan|     673|       2|
| 42|management|  single| tertiary|     no|      0|    yes| yes|  5|  dec|     562|       2|
| 56|management| married| tertiary|     no|    830|    yes| yes|  6|  oct|    1201|       1|
| 60|   retired|divorced|secondary|     no|    545|    yes|  no|  6|  

In [ ]:
from ophelia.functions import Selects
from ophelia.generic import regex_expr

bank_df.selectRegex(regex_expr(["day", "ous", "aig"])).show(5, False)

In [ ]:
bank_df.selectContains(["day", "ous", "aig"]).show(5, False)

In [ ]:
from ophelia.generic import string_match

bank_df.where(string_match("housing == yes")).crosstab("education", "campaign").orderBy(
    "education_campaign"
).show()

In [ ]:
test_agg_dict_ = {"balance": "sum"}
bank_df.where(string_match("housing == yes")).toWide(
    "education", "campaign", test_agg_dict_
).orderBy("education_campaign").show()

In [ ]:
from ophelia.functions import MapItem

map_values = {"A": "0", "B": "1", "C": "2"}

df.mapItem("Product", "bin_product", map_values).show()